# Snapshots

Qiskit Aer simulators provide a mechanism that enables you to peek inside the simulator and extract valuable data. The mechanism is named `snapshot`. Several types of snapshots are provided:
* Statevector
* Amplitudes
* Probabilities
* Density matrix
* Expectation value
* Stabilizer

Snapshots can be placed anywhere in the construction of the circuit. Snapshot parameters include the snapshot type and a label. Other parameters are optional, depending on the snapshot type. There are two such parameters: `qubits` and `params`.

### Statevector snapshot
The `statevector` snapshot gives the current state of circuit. It applies to all qubits. It accepts no additional parameters.

In [1]:
from qiskit import *
from qiskit.providers.aer import *

#Construct quantum circuit
circ = QuantumCircuit(3, 3)
circ.h(0)
circ.cx(0, 1)
circ.x(2)
circ.snapshot(label='before measure', snapshot_type = 'statevector')
circ.measure_all()
circ.snapshot(label='after measure', snapshot_type = 'statevector')
job = execute([circ], QasmSimulator(), shots=1).result()
print('Before measure = ' + str(job.data(0)['snapshots']['statevector']['before measure']))
print('After measure = ' + str(job.data(0)['snapshots']['statevector']['after measure']))

Before measure = [array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j])]
After measure = [array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])]


### Amplitudes snapshot
Sometime, we are interested in one or more specific amplitudes, and we would rather not print all $2^n$ amplitudes. This snapshot enables just that. The desired amplitudes are specified by their base-values in a list, either as hexa numbers or as integers. This snapshot too, applies to all qubits.


In [2]:
circ1 = QuantumCircuit(3, 3)
circ1.h(0)
circ1.cx(0, 1)
circ1.x(2)
circ1.snapshot(label = 'amplitude_000', snapshot_type = 'amplitudes', params = [0x0])
circ1.snapshot(label = 'amplitude_001', snapshot_type = 'amplitudes', params = [1])
circ1.snapshot(label = 'amplitude_2_3_4', snapshot_type = 'amplitudes', params = [2,3,4])
job = execute([circ1], QasmSimulator(method='matrix_product_state'), shots=1).result()
print(job.data(0)['snapshots']['amplitudes']['amplitude_000'])
print(job.data(0)['snapshots']['amplitudes']['amplitude_001'])
print(job.data(0)['snapshots']['amplitudes']['amplitude_2_3_4'])

[array([0.+0.j])]
[array([0.+0.j])]
[array([0.        +0.j, 0.        +0.j, 0.70710678+0.j])]


### Probabilities snapshot
The `probabilities` snapshot is similar to the `statevector` snapshot, but it provides probabilities rather than amplitudes. Here one may specify the requested qubits in any order. If not specified, the default is all qubits. For example:

In [3]:
circ2 = QuantumCircuit(3, 3)
circ2.h(0)
circ2.h(1)
circ2.h(2)
circ2.snapshot(label = 'prob all', snapshot_type = 'probabilities')
circ2.snapshot(label = 'prob1', snapshot_type = 'probabilities', qubits=[1])
job = execute([circ2], QasmSimulator(method='matrix_product_state'), shots=1).result()
print("All probabilities = " + str(job.data(0)['snapshots']['probabilities']['prob all']))
print("Probability of qubit 1 = " + str(job.data(0)['snapshots']['probabilities']['prob1']))

All probabilities = [{'value': {'0x0': 0.12499999999999994, '0x1': 0.12499999999999994, '0x2': 0.12499999999999994, '0x3': 0.12499999999999994, '0x4': 0.12499999999999994, '0x5': 0.12499999999999994, '0x6': 0.12499999999999994, '0x7': 0.12499999999999994}, 'memory': '0x0'}]
Probability of qubit 1 = [{'value': {'0x0': 0.4999999999999999, '0x1': 0.4999999999999999}, 'memory': '0x0'}]


### Density matrix snapshot
Returns the density matrix of the simulator state. It is possible to specify a subset of the qubits, in any order. The default is to provide the density matrix for all qubits.

In [4]:
circ3 = QuantumCircuit(3, 3)
circ3.h(0)
circ3.h(1)
circ3.h(2)
circ3.snapshot(label = 'Dens matrix', snapshot_type = 'density_matrix', qubits=[0, 1])
job = execute([circ3], QasmSimulator(), shots=1).result()
print("Density matrix = ")
print(job.data(0)['snapshots']['density_matrix']['Dens matrix'][0]['value'])

Density matrix = 
[[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]]


### Expectation value snapshot
There are two snapshots for expectation value: `expectation_value` and `expectation_value_pauli`. For `expectation_value_pauli`, we must specify the qubits and the Pauli gates by which we project the qubits. In addition, we can specify a sum of several Pauli terms. The `params` parameter, comprises 2 fields: the second is the list of Paulis, and the first is the coefficient by which this term is multiplied. In the following 3 examples, we request the expectation value by Z on qubit 0, expectation value by Y on qubit 1, and finally the product of both. Note that when several qubits are specified, the order of the Paulis is from right to left.


In [13]:
import numpy as np
circ4 = QuantumCircuit(3, 3)
circ4.x(0)
circ4.u3(np.pi/8, np.pi/8, np.pi/8, 1)
circ4.snapshot(label='expval0', snapshot_type = 'expectation_value_pauli', qubits = [0], params = [[1, 'Z']])
circ4.snapshot(label='expval1', snapshot_type = 'expectation_value_pauli', qubits = [1], params = [[1, 'Y']])
circ4.snapshot(label='expval01', snapshot_type = 'expectation_value_pauli', qubits = [0, 1], params = [[1, 'YZ']])
job = execute([circ4], QasmSimulator(), shots=1).result()
print(job.data(0)['snapshots']['expectation_value'])


{'expval01': [{'value': (-0.1464466094067262+0j), 'memory': '0x0'}], 'expval0': [{'value': (-1+0j), 'memory': '0x0'}], 'expval1': [{'value': (0.1464466094067262+0j), 'memory': '0x0'}]}


In the next example we compute the sum of expectation values, multiplied by their respective coefficients.

In [15]:
circ5 = QuantumCircuit(3, 3)
circ5.x(0)
circ5.u3(np.pi/8, np.pi/8, np.pi/8, 1)
circ5.u3(np.pi/4, np.pi/4, np.pi/4, 1)
circ5.snapshot(label='expval0', snapshot_type = 'expectation_value_pauli', 
               qubits = [0,1,2], params = [[1, 'IIZ'], [2, 'IYI'], [3, 'XXX']])

job = execute([circ5], QasmSimulator(), shots=1).result()
print(job.data(0)['snapshots']['expectation_value'])

{'expval0': [{'value': (0.570326141918013+0j), 'memory': '0x0'}]}
